### Graft Fault Attack on $SPHINCS^+$ Framework

We will try to attack $SPHINCS^+$ framework by grafting a fake tree on the main hypertree by exploiting an error generated by the signing alogirthm as it can be explained in this document : https://eprint.iacr.org/2018/102.pdf

In [1]:
from package.sphincs import Sphincs

### Instantiate a $SPHINCS^+$ Object and Setting parameters ###

We will be using :
   - Security Parameter: $n=2$
   - Winternitz Parameter: $w=16$
   - Hypertree Height: $h=4$
   - Hypertree Layers: $d=2$
   - $FORS$ Trees Number: $k=4$
   - $FORS$ Trees Height: $a=2$

In [2]:
sphincs = Sphincs()

sphincs.set_n(2)
sphincs.set_h(4)
sphincs.set_d(2)
sphincs.set_k(4)
sphincs.set_a(2)

We will suppose that a message has been signed twice while a flaw in the algorithm made the second signature with a faulty authentification path with private key _b'mo\x9c\x1bS\xd4\x05\x16'_

In [3]:
# Public key
pk = b'S\xd4\x05\x16'

In order to simplify this attack, me used the same $R$ variable so we could be sure our two signatures will use the same path of authentification

In [4]:
m = b'Not our fault if it breaks...'
sig1 = b'\x00\x00QUS\xff\x90\x94uvD\xa1\xd8\xce\xa0\xa8\xd2\x8b`9\x84\x10\x07-i\x94\xfe3\xdb\x97\x00\xc8E\xd3\xdd\x14\x15\x81n\x13\x1f\x8d\xc3?\xee\xfe\xb0\xa0puV\xb6OEH;\xfe\xe1'
sig2 = b'\x00\x00QUS\xff\x90\x94uvD\xa1\xd8\xce\xa0\xa8\xd2\x8b`9\x84\x10\x07-i\x94\xfe3\xdb\x97\x00\xc8E\xd3\xdd\x14\x15\x81n\x13\x00\x00r\x05\xca\xb3P\xfd\x11\xb1\xbbN\r\xd8H;\xfe\xe1'

If we check our signatures, we can see that $R$ is the same here, it could not be as long our message will go through the same $WOTS^+$ node in the root tree, but it's easier to see.

In [5]:
# Splitting SIGs in order to find wots message and signature
h_prime = sphincs._h_prime
len_0 = sphincs._len_0
n = sphincs._n
d = sphincs._d
a = sphincs._a
k = sphincs._k
h = sphincs._h

def sigs_from_main_sig(sig):
    sig_tab = []

    sig_tab += [sig[:n]]  # R

    sig_tab += [[]]  # SIG FORS
    for i in range(n,
                   n + k * (a + 1) * n,
                   n):
        sig_tab[1].append(sig[i:(i + n)])

    sig_tab += [[]]  # SIG Hypertree
    for i in range(n + k * (a + 1) * n,
                   n + k * (a + 1) * n + (h + d * len_0) * n,
                   n):
        sig_tab[2].append(sig[i:(i + n)])
    return sig_tab

def sig_wots_from_sig_xmss(sig):
    return sig[0:len_0]

def auth_from_sig_xmss(sig):
    return sig[len_0:]

def sigs_xmss_from_sig_ht(sig):
    sigs = []
    for i in range(0, d):
        sigs.append(sig[i * (h_prime + len_0):(i + 1) * (h_prime + len_0)])

    return sigs

splitted_sig1 = sigs_from_main_sig(sig1)
splitted_sig2 = sigs_from_main_sig(sig2)
xmss_sig1 = sigs_xmss_from_sig_ht(splitted_sig1[2])
xmss_sig2 = sigs_xmss_from_sig_ht(splitted_sig2[2])

# WOTS signature from the XMSS tree d-1 is signing the previous auth from tree d-2
print("Auth XMSS d-2 sig1: ", auth_from_sig_xmss(xmss_sig1[0])) 
print("WOTS SIG d-1  sig1: ", sig_wots_from_sig_xmss(xmss_sig1[1]))
print("Auth XMSS d-2 sig2: ", auth_from_sig_xmss(xmss_sig2[0]))
print("WOTS SIG d-1  sig2: ", sig_wots_from_sig_xmss(xmss_sig2[1]))
print()
print("Auth XMSS d-1 sig1: ", auth_from_sig_xmss(xmss_sig1[1]))
print("Auth XMSS d-1 sig2: ", auth_from_sig_xmss(xmss_sig2[1]))

Auth XMSS d-2 sig1:  [b'n\x13', b'\x1f\x8d']
WOTS SIG d-1  sig1:  [b'\xc3?', b'\xee\xfe', b'\xb0\xa0', b'pu', b'V\xb6', b'OE']
Auth XMSS d-2 sig2:  [b'n\x13', b'\x00\x00']
WOTS SIG d-1  sig2:  [b'r\x05', b'\xca\xb3', b'P\xfd', b'\x11\xb1', b'\xbbN', b'\r\xd8']

Auth XMSS d-1 sig1:  [b'H;', b'\xfe\xe1']
Auth XMSS d-1 sig2:  [b'H;', b'\xfe\xe1']


As we can see, our second signature has a faulted authentification path, different from the first signature. Because of this, we obtain an other signature from the same $WOTS^+$ leaf. But even if the signature is different, because it has been successfully signed, it is counted as a true signature:

In [6]:
print("Is sig1 correct? ", sphincs.verify(sig1, m, pk))
print("Is sig2 correct? ", sphincs.verify(sig2, m, pk))

Is sig1 correct?  True
Is sig2 correct?  True


We need to find a random _seed_ for which our forged message go through the same $WOTS^+$ leaf. To do this, we need to find a $R$ that the verifying algorithm will use to go through the attacked leaf.

In [7]:
m_fake = b'The earth is flat!'

# Here for this hypertree and this fake message, we know that if R = b'a\x06', we will go through the same leaf on d-1 tree
r = b'a\x06'

This value has been easily found because we can still sign our own message with the secret key (we could then obtain the message signature: b'a\x06\xe1\xe6\xea\xfb\xbd)\xd0+\x9b\xd7+\x94\xd8Vk\xcct\x9fzl\xf1\xe4\xb1%\xcfD\xf4\xa5\xa8\x1e\x14\xa7#\x1a\xe2\xef\xd9\x08\x1f\x8d\xc3?\xee\xfe\xb0\xa0puV\xb6OEH;\xfe\xe1'). But knowing which path is used to verify the signature is publicly known: it depends from $pk$, $m$ and $R$ which are publicly known when we digest the message in the first verification step.

From these informations, we need to compute the $m_{fake}$ signature with different sub-hypertrees (size $h - h'$ with $d-1$ layers) until we can found a tree for which his computed root can be signed with the known $WOTS^+$ leaf. The more faulted signature we have, the more possible roots can be forged. 

From this part, we have to brute force sub-hypertrees possibilities until we found one that can be used with this $WOTS^+$ key. We can then sign our message with our $R$, our tree and we then add the final official authentification path to the signature to make it computable by others.